In [1]:
from dash import Dash, dcc, html, Input, Output, State
import plotly.express as px
import pandas as pd
import numpy as np

In [2]:
app = Dash(__name__)
app.title = "Социально-финансовый мониторинг"

In [3]:
regions = ['Регион 1', 'Регион 2', 'Регион 3', 'Регион 4', 'Регион 5']
years = list(range(2000, 2021))

In [4]:
data = {
    "Регион": np.random.choice(regions, 100),
    "Год": np.random.choice(years, 100),
    "ВРП": np.random.uniform(100, 1000, 100),
    "Индекс развития": np.random.uniform(0.5, 1.5, 100),
}
df = pd.DataFrame(data)

# Функция для фильтрации данных по региону
def filter_region_data(region):
    return df[df["Регион"] == region]

In [5]:
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),  # Для управления страницами
    html.Div(id='page-content')  # Контейнер для контента страниц
])

main_page = html.Div([
    html.H1("Социально-финансовый мониторинг", style={'text-align': 'center'}),
    html.Div([
        dcc.Link("Перейти к показателям региона", href='/region', style={'margin-right': '20px'}),
        dcc.Link("Перейти к индексу развития", href='/index'),
    ], style={'text-align': 'center'}),
])

region_page = html.Div([
    html.H2("Основные показатели региона", style={'text-align': 'center'}),
    html.Div([
        html.Label("Выберите регион:"),
        dcc.Dropdown(
            id='region-dropdown',
            options=[{'label': reg, 'value': reg} for reg in regions],
            value=regions[0]
        ),
    ]),
    dcc.Graph(id='region-graph'),
    html.Br(),
    dcc.Link("Вернуться на главную", href='/')
])

index_page = html.Div([
    html.H2("Динамика индекса развития", style={'text-align': 'center'}),
    html.Div([
        html.Label("Выберите регион:"),
        dcc.Dropdown(
            id='index-region-dropdown',
            options=[{'label': reg, 'value': reg} for reg in regions],
            value=regions[0]
        ),
    ]),
    dcc.Graph(id='index-graph'),
    html.Br(),
    dcc.Link("Вернуться на главную", href='/')
])

@app.callback(Output('page-content', 'children'), Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/region':
        return region_page
    elif pathname == '/index':
        return index_page
    else:
        return main_page

@app.callback(Output('region-graph', 'figure'), Input('region-dropdown', 'value'))
def update_region_graph(selected_region):
    filtered_data = filter_region_data(selected_region)
    fig = px.line(filtered_data, x="Год", y="ВРП", title=f"Динамика ВРП для {selected_region}")
    fig.update_traces(mode='lines+markers')
    return fig

@app.callback(Output('index-graph', 'figure'), Input('index-region-dropdown', 'value'))
def update_index_graph(selected_region):
    filtered_data = filter_region_data(selected_region)
    similar_regions = df[df["Год"].isin(filtered_data["Год"].unique())].groupby("Регион").mean()
    top_similar = similar_regions["Индекс развития"].nlargest(6).reset_index()  # Топ-5 + выбранный регион
    
    fig = px.line(filtered_data, x="Год", y="Индекс развития", title=f"Индекс развития для {selected_region}")
    for region in top_similar["Регион"]:
        if region != selected_region:
            similar_data = df[df["Регион"] == region]
            fig.add_scatter(x=similar_data["Год"], y=similar_data["Индекс развития"], mode='lines', name=region)
    return fig

In [11]:
if __name__ == '__main__':
    app.run_server(debug=True)